In [257]:
import tensorflow as tf
import numpy as np

DATA

In [258]:
import tensorflow_datasets as tfds

In [259]:
mnist_dataset , mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

In [260]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

In [261]:
num_validation_samples=0.1*mnist_info.splits['train'].num_examples

print(num_validation_samples)

6000.0


In [262]:
print(mnist_info.splits['train'].num_examples)

60000


FUNCTION FOR SCALING

In [263]:
def scale(image,label):
  image=tf.cast(image,tf.float32)
  image/=255.
  return image,label

SCALING

In [264]:
scaled_train_and_validation_data=mnist_train.map(scale)

In [265]:
test_data=mnist_test.map(scale)

In [266]:
BUFFER_SIZE=10000
shuffled_train_and_validation=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

SPLITTING TRAINING DATA SET TO VALIDATION AND TRAINING

In [267]:
num_validation_samples=tf.cast(num_validation_samples,tf.int64)
validation_data=shuffled_train_and_validation.take(num_validation_samples)

In [268]:
train_data=shuffled_train_and_validation.skip(num_validation_samples)


BATCHING DATA SET
 
BATCH SIZE CAN BE FINE TUNED.

In [269]:
BATCH_SIZE=1000
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_num_samples=mnist_info.splits['test'].num_examples
test_num_samples=tf.cast(test_num_samples,tf.int64)
test_data=test_data.batch(test_num_samples)


In [270]:
validation_inputs, validation_targets=next(iter(validation_data))

In [271]:
#train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

MODEL DESIGN

In [272]:
input_size=784
output_size=10
hidden_layer_size=300

model=tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size,activation='relu'),  
        tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size,activation='relu'), 
        tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size,activation='tanh'), 
        tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
        tf.keras.layers.Dense(output_size,activation='softmax')
        


    ]





)

OPTIMIZER & LOSS FUNCTION

In [273]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

TRAINING

In [274]:
num_epochs=5

model.fit(train_data,epochs=num_epochs,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
54/54 - 12s - loss: 0.5103 - accuracy: 0.8421 - val_loss: 0.1661 - val_accuracy: 0.9543 - 12s/epoch - 220ms/step
Epoch 2/5
54/54 - 8s - loss: 0.1420 - accuracy: 0.9561 - val_loss: 0.1144 - val_accuracy: 0.9633 - 8s/epoch - 153ms/step
Epoch 3/5
54/54 - 8s - loss: 0.0943 - accuracy: 0.9710 - val_loss: 0.0821 - val_accuracy: 0.9737 - 8s/epoch - 148ms/step
Epoch 4/5
54/54 - 8s - loss: 0.0606 - accuracy: 0.9808 - val_loss: 0.0631 - val_accuracy: 0.9805 - 8s/epoch - 156ms/step
Epoch 5/5
54/54 - 8s - loss: 0.0503 - accuracy: 0.9841 - val_loss: 0.0481 - val_accuracy: 0.9860 - 8s/epoch - 141ms/step


TESTING THE MODEL

In [276]:
test_loss ,test_accuracy = model.evaluate(test_data)

#print('test loss: {0:.2f}. test accuracy: {1:.2f}% .format(test_loss,test_accuracy*100.))

1/1 [==============================] - 1s 1s/step - loss: 0.0803 - accuracy: 0.9755


In [277]:
print(test_accuracy)

0.9754999876022339


In [286]:
print('test accuracy: {0:.2f}%' .format(test_accuracy*100.) )

test accuracy: 97.55%
